In [1]:
import numpy as np
import random
import math
import time
import librosa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as T
import torch.optim as optim
import torchaudio

import IPython.display as ipd
from tqdm.notebook import tqdm

In [2]:
# 定义自己的数据集
# 使用pandas 连接label下的文件
talser_path='D:/data/TAL-SER/'
label_df = pd.read_csv(talser_path+"label/label",sep=" ")
utt2gen_df = pd.read_csv(talser_path+"label/utt2gen",sep=" ",header=None,names=['id','sex'])
utt2spk_df = pd.read_csv(talser_path+"label/utt2spk",sep=" ",header=None,names=['id','speaker'])
wavscp_df = pd.read_csv(talser_path+"label/wavscp",sep=" ",header=None,names=['id','path'])
wavscp_df.sort_values('id', ignore_index=True,inplace =True)
data_df = pd.concat([label_df, utt2spk_df['speaker'], utt2gen_df['sex'],wavscp_df['path']], axis=1)
# data_df["path"].replace(".\/S", talser_path+"S", regex=True,inplace =True)
del label_df
del utt2gen_df
del utt2spk_df
del wavscp_df

def function(P, A):
    if  P>0:
        if A>0:
            return 'AH'
        else:
            return 'PH'
    else:
        if A<0:
            return 'AL'
        else:
            return 'PL'
data_df['emotion'] = data_df.apply(lambda x: function(x.P, x.A), axis = 1)

data_df.sample()

,id,P,A,speaker,sex,path,emotion
4059,6607,-0.434956,0.939179,SER0030,female,./SER0030/6607.wav,PL


In [5]:
import torch
try:
    from torch_audiomentations import Compose, Gain, PolarityInversion
except:
    !pip install torch-audiomentations


# Initialize augmentation callable
apply_augmentation = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)

torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make an example tensor with white noise.
# This tensor represents 8 audio snippets with 2 channels (stereo) and 2 s of 16 kHz audio.
audio_samples = torch.rand(size=(8, 2, 32000), dtype=torch.float32, device=torch_device) - 0.5

# Apply augmentation. This varies the gain and polarity of (some of)
# the audio snippets in the batch independently.
perturbed_audio_samples = apply_augmentation(audio_samples, sample_rate=16000)